### **Librerias** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

### **Extraccion datos ES** ###

In [1]:

import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass
class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato ES correcto (GLOBEX)contrato.symbol = "BP"===========
contrato = Contract()
contrato.symbol = "ES"
contrato.secType = "FUT"
contrato.exchange = "CME"
contrato.currency = "USD"
contrato.lastTradeDateOrContractMonth = "202512" 
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()  # usar UTC
all_dataframes = []

for i in range(1):  # 4 bloques de 1 mes
    end_str = end.strftime("%Y%m%d-%H:%M:%S")  # formato con guion y UTC

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="3600 S",          # máximo permitido para 4 meses
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp['Date'] = pd.to_datetime(df_temp['Date'], format='%Y%m%d %H:%M:%S', utc=True, errors='coerce')
        all_dataframes.append(df_temp)

    # Retroceder un mes para la siguiente solicitud
    end = end - timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values('Date').drop_duplicates(subset=['Date'])

    for c in ['Open', 'High', 'Low', 'Close', 'Volume', 'Count']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    # Exportar CSV
    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_ES.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251128  19:33:06 -> 20251128  20:33:06
✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entr

### **Extraccion datos EC** ###

In [ ]:
import threading
import pandas as pd
from datetime import datetime, timedelta
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os
import getpass

# ============================================================
#   CLASE IB PARA DATOS HISTÓRICOS
# ============================================================
class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin datos históricos -> reqId={reqId}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")
        self.evento.set()

    # Para verificar que el contrato existe
    def contractDetails(self, reqId, details):
        print("📌 Contrato encontrado:", details.contract)


# ============================================================
#   INICIAR CONEXIÓN
# ============================================================
IB = IB_DatosHistoricos_Futuros()
IB.connect("127.0.0.1", 7497, clientId=1)

api_thread = threading.Thread(target=IB.run, daemon=True)
api_thread.start()

IB.evento.wait()
IB.evento.clear()


# ============================================================
#   CONTRATO FUTURO **EC (Euro FX, CME)**
# ============================================================
contrato = Contract()
contrato.symbol = "EUR"                        # subyacente Euro FX
contrato.secType = "FUT"                       # tipo de contrato
contrato.exchange = "CME"                      # exchange correcto
contrato.currency = "USD"                      # divisa
contrato.tradingClass = "EC"                   # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)

# ============================================================
#   VERIFICAR CONTRATO
# ============================================================
IB.reqContractDetails(99, contrato)
IB.evento.wait()
IB.evento.clear()

# ============================================================
#   SOLICITUD DE DATOS (4 MESES)
# ============================================================
end = datetime.utcnow()
all_data = []

for req in range(1, 5):

    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB.reqHistoricalData(
        reqId=req,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB.evento.wait()
    IB.evento.clear()

    if req in IB.datos_historicos:
        df_temp = pd.DataFrame(IB.datos_historicos[req])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_data.append(df_temp)

    end -= timedelta(days=30)


# ============================================================
#   UNIR + LIMPIAR + EXPORTAR CSV
# ============================================================
if all_data:
    df = pd.concat(all_data, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    numeric_cols = ["Open", "High", "Low", "Close", "Volume", "Count"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

    print("✅ DataFrame final listo!")
    print(df.head())

    usuario = getpass.getuser()
    ruta_csv = os.path.join("C:\\Users", usuario, "Downloads", "historico_EC.csv")
    df.to_csv(ruta_csv, index=False)

    print(f"📁 Archivo guardado en:\n{ruta_csv}")

else:
    print("❗ No se recibieron datos.")


IB.disconnect()


### **Extraccion datos NQ** ###

In [ ]:

import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass
class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato NQ correcto (GLOBEX)
contrato = Contract()
contrato.symbol = "NQ"
contrato.secType = "FUT"
contrato.exchange = "CME"
contrato.currency = "USD"
contrato.lastTradeDateOrContractMonth = "202512" 
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()  # usar UTC
all_dataframes = []

for i in range(4):  # 4 bloques de 1 mes
    end_str = end.strftime("%Y%m%d-%H:%M:%S")  # formato con guion y UTC

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",          # máximo permitido para 4 meses
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp['Date'] = pd.to_datetime(df_temp['Date'], format='%Y%m%d %H:%M:%S', utc=True, errors='coerce')
        all_dataframes.append(df_temp)

    # Retroceder un mes para la siguiente solicitud
    end = end - timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values('Date').drop_duplicates(subset=['Date'])

    for c in ['Open', 'High', 'Low', 'Close', 'Volume', 'Count']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    # Exportar CSV
    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_NQ.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

IB_conexion.disconnect()


### **Extraccion datos YM** ###

In [ ]:

import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass
class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato YM correcto (GLOBEX)
contrato = Contract()
contrato.symbol = "DJIA"                       # subyacente Dow Jones Industrial Average
contrato.secType = "FUT"                       # tipo de contrato: FUT
contrato.exchange = "CBOT"                     # exchange correcto (Chicago Board of Trade, parte de CME)
contrato.currency = "USD"                      # divisa
contrato.tradingClass = "YM"                   # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()  # usar UTC
all_dataframes = []

for i in range(4):  # 4 bloques de 1 mes
    end_str = end.strftime("%Y%m%d-%H:%M:%S")  # formato con guion y UTC

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",          # máximo permitido para 4 meses
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp['Date'] = pd.to_datetime(df_temp['Date'], format='%Y%m%d %H:%M:%S', utc=True, errors='coerce')
        all_dataframes.append(df_temp)

    # Retroceder un mes para la siguiente solicitud
    end = end - timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values('Date').drop_duplicates(subset=['Date'])

    for c in ['Open', 'High', 'Low', 'Close', 'Volume', 'Count']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    # Exportar CSV
    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_NQ.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

IB_conexion.disconnect()


### **Extraccion datos BP** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO 6B (British Pound, CME)
# =============================
contrato = Contract()
contrato.symbol = "GBP"                         # subyacente British Pound
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CME"                       # exchange correcto
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "6B"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)

# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_6B.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos EC** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "EUR"                        # símbolo raíz correcto
contrato.secType = "FUT"
contrato.exchange = "CME"                      # CME Globex
contrato.currency = "USD"
contrato.tradingClass = "6E"                   # clase de trading
contrato.lastTradeDateOrContractMonth = "202603"  # vencimiento real (ejemplo marzo 2026)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_EC.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251025  14:29:23 -> 20251125  14:29:23


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_5760\1984362956.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250926  14:29:23 -> 20251026  14:29:23


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_5760\1984362956.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250826  14:29:23 -> 20250926  14:29:23


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_5760\1984362956.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250727  14:29:23 -> 20250827  14:29:23


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_5760\1984362956.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 98800 entries, 80967 to 28334
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    98800 non-null  datetime64[ns, UTC]
 1   Open    98800 non-null  float64            
 2   High    98800 non-null  float64            
 3   Low     98800 non-null  float64            
 4   Close   98800 non-null  float64            
 5   Volume  98800 non-null  int64              
 6   Count   98800 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 6.0 MB
None
                           Date     Open     High      Low    Close  Volume  \
80967 2025-07-29 13:15:00+00:00  1.16895  1.16895  1.16895  1.16895       3   
80968 2025-07-29 13:16:00+00:00  1.16900  1.16900  1.16895  1.16895      17   
80969 2025-07-29 13:17:00+00:00  1.16895  1.16895  1.16895  1.16895       0   
80970 2025-07-29 13:1

: 

### **Extraccion datos GC** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "GC"                          # subyacente Gold Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "COMEX"                     # exchange correcto (parte de CME Group)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "GC"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_6C.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos HG** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "HG"                          # subyacente Copper Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "COMEX"                     # exchange correcto (parte de CME Group)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "HG"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_HG.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos HO** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "HO"                          # subyacente Heating Oil Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "NYMEX"                     # exchange correcto (parte de CME Group)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "HO"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_HG.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos JY** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "6J"                        # subyacente Japanese Yen
contrato.secType = "FUT"                       # tipo de contrato: FUT
contrato.exchange = "CME"                      # exchange correcto (CME Globex)
contrato.currency = "USD"                      # divisa
contrato.tradingClass = "6J"                   # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_JY.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos NG** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "NG"                          # subyacente Natural Gas Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "NYMEX"                     # exchange correcto (parte de CME Group)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "NG"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_JY.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos NZ** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "6N"                        # subyacente New Zealand Dollar
contrato.secType = "FUT"                       # tipo de contrato: FUT
contrato.exchange = "CME"                      # exchange correcto (CME Globex)
contrato.currency = "USD"                      # divisa
contrato.tradingClass = "6N"                   # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_NZ.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos MBT** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "MBT"                        # Micro Bitcoin Futures
contrato.secType = "FUT"                       # tipo de contrato: FUT
contrato.exchange = "CME"                      # exchange correcto (CME Globex)
contrato.currency = "USD"                      # divisa
contrato.tradingClass = "MBT"                  # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_MBT.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos RB** ###

In [2]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass
import time


class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.error_en_req = {}  # <<--- REGISTRA SI HUBO ERROR EN UN REQID
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        print("🔌 Conexión válida con IBKR. ID listo.")
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")

        # SI HAY ERROR 162 → NO HAY DATOS. NO DETENER PROCESO.
        if code == 162:
            print("⚠ No hay datos para este mes. Se continúa con el siguiente.")
            self.error_en_req[reqId] = True
            self.evento.set()  # Desbloquear espera para este reqId
        else:
            self.error_en_req[reqId] = True
            self.evento.set()


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()


# =============================
# Contrato FUTURO RB
# =============================
contrato = Contract()
contrato.symbol = "RB"
contrato.secType = "FUT"
contrato.exchange = "NYMEX"
contrato.currency = "USD"
contrato.tradingClass = "RB"
# contrato.lastTradeDateOrContractMonth = ""  # opcional — dejar vacío para front month


# =============================
# DESCARGA DESDE 2008 — MES POR MES
# =============================
all_dataframes = []

year_start = 2008
current_year = datetime.utcnow().year
today = datetime.utcnow()

req_id = 1

for year in range(year_start, current_year + 1):
    for month in range(1, 13):

        if year == today.year and month > today.month:
            break

        print(f"\n📌 Extrayendo datos del mes {month:02d}/{year}…")

        # Fecha final del mes
        fecha_final = datetime(year, month, 28) + timedelta(days=5)
        fecha_final = fecha_final.replace(day=1) - timedelta(days=1)
        fecha_final_str = fecha_final.strftime("%Y%m%d-%H:%M:%S")

        IB_conexion.error_en_req[req_id] = False
        IB_conexion.datos_historicos[req_id] = []

        IB_conexion.reqHistoricalData(
            reqId=req_id,
            contract=contrato,
            endDateTime=fecha_final_str,
            durationStr="1 M",
            barSizeSetting="1 day",
            whatToShow="TRADES",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[]
        )

        IB_conexion.evento.wait()
        IB_conexion.evento.clear()

        if IB_conexion.error_en_req[req_id]:
            print(f"⛔ No hubo datos válidos en {month:02d}/{year}. Continuando…")
            req_id += 1
            continue

        datos = IB_conexion.datos_historicos.get(req_id, [])
        if datos:
            df_temp = pd.DataFrame(datos)
            df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
            all_dataframes.append(df_temp)

            print(f"✔ Datos obtenidos para {month:02d}/{year}.")
        else:
            print(f"⚠ Mes sin datos: {month:02d}/{year}")

        req_id += 1
        time.sleep(1)


# =============================
# Concatenar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("\n✅ Datos finales:")
    print(df.info())

    usuario = getpass.getuser()
    ruta = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_RB.csv")
    df.to_csv(ruta, index=False)

    print(f"📁 Archivo guardado en: {ruta}")

else:
    print("❗ No se recibieron datos.")


IB_conexion.disconnect()


🔌 Conexión válida con IBKR. ID listo.

📌 Extrayendo datos del mes 01/2008…
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
⚠ Mes sin datos: 01/2008
❗ Error reqId=1, code=321, msg=Error validating request.-'bL' : cause - Please e

### **Extraccion datos KW** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "KE"                          # Hard Red Winter Wheat Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CBOT"                      # exchange correcto (Chicago Board of Trade)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "KE"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_KW.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos LC** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "LE"                          # subyacente Live Cattle Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CME"                       # exchange correcto (CME Globex)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "LE"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_LC.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos LH** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "HE"                          # subyacente Lean Hogs Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CME"                       # exchange correcto (CME Globex)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "HE"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_LH.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos RTY** ###

In [ ]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "RTY"                         # subyacente E-mini Russell 2000 Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CME"                       # exchange correcto (CME Globex)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "RTY"                   # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_RTY.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


### **Extraccion datos SF** ###

In [1]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "6S"                          # Swiss Franc Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CME"                       # exchange correcto (CME Globex)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "6S"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_SF.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251024  22:47:39 -> 20251124  22:47:39


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\1191163499.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250925  22:47:39 -> 20251025  22:47:39


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\1191163499.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250825  22:47:39 -> 20250925  22:47:39


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\1191163499.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250726  22:47:39 -> 20250826  22:47:39


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\1191163499.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 111949 entries, 87842 to 28979
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype              
---  ------  --------------   -----              
 0   Date    111949 non-null  datetime64[ns, UTC]
 1   Open    111949 non-null  float64            
 2   High    111949 non-null  float64            
 3   Low     111949 non-null  float64            
 4   Close   111949 non-null  float64            
 5   Volume  111949 non-null  int64              
 6   Count   111949 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 6.8 MB
None
                           Date   Open   High    Low  Close  Volume  Count
87842 2025-07-29 04:18:00+00:00  1.265  1.265  1.265  1.265       1      1
87843 2025-07-29 04:19:00+00:00  1.265  1.265  1.265  1.265       0      0
87844 2025-07-29 04:20:00+00:00  1.265  1.265  1.265  1.265       0      0
87845 2025-07-29 04:21:00+0

### **Extraccion datos SI** ###

In [2]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "SI"                          # Silver Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "COMEX"                     # exchange correcto (parte de CME Group)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "SI"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_SI.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251024  22:00:36 -> 20251124  22:00:36


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2428\3088976116.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250925  22:00:36 -> 20251025  22:00:36


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2428\3088976116.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250825  22:00:36 -> 20250925  22:00:36


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2428\3088976116.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250726  22:00:36 -> 20250826  22:00:36


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2428\3088976116.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 115605 entries, 88162 to 28970
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype              
---  ------  --------------   -----              
 0   Date    115605 non-null  datetime64[ns, UTC]
 1   Open    115605 non-null  float64            
 2   High    115605 non-null  float64            
 3   Low     115605 non-null  float64            
 4   Close   115605 non-null  float64            
 5   Volume  115605 non-null  int64              
 6   Count   115605 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 7.1 MB
None
                           Date    Open    High     Low   Close  Volume  Count
88162 2025-07-28 22:03:00+00:00  38.805  38.805  38.805  38.805       1      1
88163 2025-07-28 22:04:00+00:00  38.805  38.805  38.805  38.805       0      0
88164 2025-07-28 22:05:00+00:00  38.805  38.805  38.805  38.805       0      0
88165 2025-

### **Extraccion datos ZC** ###

In [2]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "ZC"                          # Corn Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CBOT"                      # exchange correcto (Chicago Board of Trade)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "ZC"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_ZC.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251024  23:23:15 -> 20251124  23:23:15


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\3606620764.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250925  23:23:15 -> 20251025  23:23:15


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\3606620764.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250825  23:23:15 -> 20250925  23:23:15


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\3606620764.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250726  23:23:15 -> 20250826  23:23:15


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\3606620764.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 89675 entries, 67520 to 22154
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    89675 non-null  datetime64[ns, UTC]
 1   Open    89675 non-null  float64            
 2   High    89675 non-null  float64            
 3   Low     89675 non-null  float64            
 4   Close   89675 non-null  float64            
 5   Volume  89675 non-null  int64              
 6   Count   89675 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 5.5 MB
None
                           Date    Open    High     Low   Close  Volume  Count
67520 2025-07-28 00:00:00+00:00  419.00  419.00  417.50  417.75    2245    249
67521 2025-07-28 00:01:00+00:00  417.50  417.75  417.50  417.50     259     42
67522 2025-07-28 00:02:00+00:00  417.75  418.25  417.75  417.75     122     15
67523 2025-07-28 00:0

### **Extraccion datos ZL** ###

In [3]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "ZL"                          # Soybean Oil Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CBOT"                      # exchange correcto (Chicago Board of Trade)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "ZL"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_ZL.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251025  03:30:27 -> 20251125  03:30:27


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2031540465.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250926  03:30:27 -> 20251026  03:30:27


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2031540465.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250826  03:30:27 -> 20250926  03:30:27


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2031540465.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250727  03:30:27 -> 20250827  03:30:27


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2031540465.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 87916 entries, 66816 to 22294
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    87916 non-null  datetime64[ns, UTC]
 1   Open    87916 non-null  float64            
 2   High    87916 non-null  float64            
 3   Low     87916 non-null  float64            
 4   Close   87916 non-null  float64            
 5   Volume  87916 non-null  int64              
 6   Count   87916 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 5.4 MB
None
                           Date   Open   High    Low  Close  Volume  Count
66816 2025-07-29 00:00:00+00:00  55.99  55.99  55.89  55.96     291    124
66817 2025-07-29 00:01:00+00:00  55.99  56.00  55.97  55.97     109     42
66818 2025-07-29 00:02:00+00:00  55.98  56.02  55.97  56.02      31     12
66819 2025-07-29 00:03:00+00:00  56.0

### **Extraccion datos ZM** ###

In [4]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "ZM"                          # Soybean Meal Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CBOT"                      # exchange correcto (Chicago Board of Trade)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "ZM"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202512"  # vencimiento (ejemplo diciembre 2025)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_ZL.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251025  03:57:38 -> 20251125  03:57:38


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2378950391.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250926  03:57:38 -> 20251026  03:57:38


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2378950391.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250826  03:57:38 -> 20250926  03:57:38


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2378950391.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250727  03:57:38 -> 20250827  03:57:38


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_17092\2378950391.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 87971 entries, 66871 to 22322
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    87971 non-null  datetime64[ns, UTC]
 1   Open    87971 non-null  float64            
 2   High    87971 non-null  float64            
 3   Low     87971 non-null  float64            
 4   Close   87971 non-null  float64            
 5   Volume  87971 non-null  int64              
 6   Count   87971 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 5.4 MB
None
                           Date   Open   High    Low  Close  Volume  Count
66871 2025-07-29 00:00:00+00:00  279.3  279.9  279.0  279.3     494    161
66872 2025-07-29 00:01:00+00:00  279.2  279.2  279.2  279.2      27     11
66873 2025-07-29 00:02:00+00:00  279.2  279.2  279.2  279.2       0      0
66874 2025-07-29 00:03:00+00:00  279.

### **Extraccion datos ZS** ###

In [1]:
import threading
from datetime import datetime, timedelta
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import os, getpass

class IB_DatosHistoricos_Futuros(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.datos_historicos = {}
        self.objetos_datos = {}
        self.evento = threading.Event()

    def nextValidId(self, orderId):
        self.evento.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.datos_historicos:
            self.datos_historicos[reqId] = []
            self.objetos_datos[reqId] = []

        self.datos_historicos[reqId].append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume,
            "Count": bar.barCount
        })
        self.objetos_datos[reqId].append(bar)

    def historicalDataEnd(self, reqId, start, end):
        print(f"✅ Fin de datos históricos para ID: {reqId} | Rango: {start} -> {end}")
        self.evento.set()

    def error(self, reqId, code, msg):
        print(f"❗ Error reqId={reqId}, code={code}, msg={msg}")


# =============================
# Conexión
# =============================
IB_conexion = IB_DatosHistoricos_Futuros()
IB_conexion.connect(host="127.0.0.1", port=7497, clientId=1)

api_thread = threading.Thread(target=IB_conexion.run, daemon=True)
api_thread.start()

IB_conexion.evento.wait()
IB_conexion.evento.clear()

# =============================
# Contrato FUTURO GC (Gold, CME)
# =============================
contrato = Contract()
contrato.symbol = "ZS"                          # Soybeans Futures
contrato.secType = "FUT"                        # tipo de contrato: FUT
contrato.exchange = "CBOT"                      # exchange correcto (Chicago Board of Trade)
contrato.currency = "USD"                       # divisa
contrato.tradingClass = "ZS"                    # clase de trading
contrato.lastTradeDateOrContractMonth = "202603"  # vencimiento real (ejemplo marzo 2026)
# =============================
# Solicitud en bloques de 1 mes
# =============================
end = datetime.utcnow()
all_dataframes = []

for i in range(4):
    end_str = end.strftime("%Y%m%d-%H:%M:%S")

    IB_conexion.reqHistoricalData(
        reqId=i+1,
        contract=contrato,
        endDateTime=end_str,
        durationStr="1 M",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,   # timestamps UNIX UTC
        keepUpToDate=False,
        chartOptions=[]
    )

    IB_conexion.evento.wait()
    IB_conexion.evento.clear()

    if (i+1) in IB_conexion.datos_historicos:
        df_temp = pd.DataFrame(IB_conexion.datos_historicos[i+1])
        df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)
        all_dataframes.append(df_temp)

    end -= timedelta(days=30)

# =============================
# Concatenar y limpiar
# =============================
if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    df = df.sort_values("Date").drop_duplicates(subset=["Date"])

    for c in ["Open", "High", "Low", "Close", "Volume", "Count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    print("✅ Datos concatenados correctamente:")
    print(df.info())
    print(df.head())

    usuario = getpass.getuser()
    ruta_descargas = os.path.join("C:\\Users", usuario, "Downloads", "historico_futuros_ZS.csv")
    df.to_csv(ruta_descargas, index=False)
    print(f"📁 Archivo guardado en: {ruta_descargas}")

else:
    print("❗ No se recibieron datos.")

IB_conexion.disconnect()


❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
❗ Error reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
❗ Error reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
❗ Error reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefnj
✅ Fin de datos históricos para ID: 1 | Rango: 20251025  04:15:56 -> 20251125  04:15:56


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_10124\2974026431.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 2 | Rango: 20250926  04:15:56 -> 20251026  04:15:56


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_10124\2974026431.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 3 | Rango: 20250826  04:15:56 -> 20250926  04:15:56


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_10124\2974026431.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Fin de datos históricos para ID: 4 | Rango: 20250727  04:15:56 -> 20250827  04:15:56


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_10124\2974026431.py:92: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_temp["Date"] = pd.to_datetime(df_temp["Date"], unit="s", utc=True)


✅ Datos concatenados correctamente:
<class 'pandas.core.frame.DataFrame'>
Index: 88007 entries, 66907 to 22340
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    88007 non-null  datetime64[ns, UTC]
 1   Open    88007 non-null  float64            
 2   High    88007 non-null  float64            
 3   Low     88007 non-null  float64            
 4   Close   88007 non-null  float64            
 5   Volume  88007 non-null  int64              
 6   Count   88007 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2)
memory usage: 5.4 MB
None
                           Date     Open     High      Low    Close  Volume  \
66907 2025-07-29 00:00:00+00:00  1044.75  1046.25  1044.75  1045.50      55   
66908 2025-07-29 00:01:00+00:00  1045.75  1046.25  1045.75  1045.75      10   
66909 2025-07-29 00:02:00+00:00  1045.75  1045.75  1045.75  1045.75       0   
66910 2025-07-29 00:0